# loop over waveforms from each station

### import the catalog

### 1) for list of stations that may be used
### 2) find how many events have data at each station
### 3) test for clipping
### 4) write out lists results for each event and station:  0=none, 1=good, 2=clipped


In [1]:
import numpy as np
from matplotlib import pyplot as plt
import datetime
import pandas as pd
import os
import glob
import obspy

In [21]:
DirCatalog = '/home/ben/data/Kilauea/'
cat_orig = 'EQ_Eruption_Catalog_4_1_2018_8_30_2018_HVO.csv'

FOV='zoomOnecycle_v2'

if FOV=='big':
    station_name = 'STCD'
    catname_pkl = './cat_STDC_bigview.pkl'  
    
if FOV=='zoom':
    station_name = 'PUHI'
    catname_pkl = './catalogs/cat_PUHI_zoomview.pkl'  

if FOV=='zoomOnecycle_v2':
    station_name = 'PUHI'
    catname_pkl = './catalogs/cat_PUHI_zoomOnecycle_v2.pkl'  
    
DataDir = '/home/ben/data/Kilauea/waveforms_' + station_name + '/'


In [22]:
cat = pd.read_csv(DirCatalog+cat_orig,header=0, encoding='utf_8')
#date_time_str_list = catalog.iloc[:,0].tolist()
print('The catalog has events: ' + str(cat.shape[0]))
print(cat.columns)

print(cat.id[0:5])

#names_vec = ['time','lat','lon','dep','mag','degrees','id', 'type', 'error1', 'error2', 'error3','valid']
#tab = pd.read_csv(DirCatalog + FileCatalog, header=None, delimiter=',', names=names_vec)
        

#print('The catalog is a dataframe of size ' + str(tab.shape))
#print('I will assume (correct me if I\'m wrong) that the fields are:\n\t year\n\t month\n\t day\n\t hour\n\t minute\n\t seconds\n\t latitude\n\t longitude\n\t depth\n\t magnitude\n\t id')
#print('There are ' + str(len(tab['time'])) + ' events')

The catalog has events: 41328
Index(['time', 'latitude', 'longitude', 'depth', 'mag',
       'dist from Halemaumau (degrees)', 'id', 'type', 'horizontalError',
       'depthError', 'magError'],
      dtype='object')
0    hv70068046
1    hv70068071
2    hv70068076
3    hv70068091
4    hv70068096
Name: id, dtype: object


In [23]:
# NEED TO FILTER EVENTS BY VIEW AND TIME WINDOW FOR ZOOM IF MAKING ZOOM ! 
print('number of events, pre-filt: ' + str(len(cat)))

starttimes = np.zeros(len(cat['time']))
for k in range(0,len(cat['time'])-1):
    starttimes[k] = obspy.UTCDateTime(cat['time'][k]).timestamp
    
cat['starttimes'] = pd.Series(starttimes) # and put it in the dataframe
cat = cat[cat.starttimes>0.0]

cat = cat.sort_values(by=['starttimes'])
# And reset indices
cat = cat.reset_index()



number of events, pre-filt: 41328


In [52]:
# trial and error !
print('number of events, pre-filt: ' + str(len(cat)))

# start_time = pd.Timestamp(2018,6,2,3,43,36,0)
# print(start_time)
# print(cat.columns)
# start_idx = cat.time[cat.time == start_time]
# print(start_idx)
# print(type(start_time))
# print(type(cat.time[0]))

#start_day = pd.to_datetime('2018-05-31')

#  2018,6,2,3,43,36,0

#  2018,6,4,7,50,59,0


start_day = pd.to_datetime('2018-06-02')
#print('start day = ' + str(start_day))
print(cat.time[5354])

2018,6,4,7,50,59,0
#end_day = pd.to_datetime('2018-08-08')
end_day = pd.to_datetime('2018-06-04')
#print('end day = ' + str(end_day))
print(cat.time[5933])

#pd.to_datetime(tab.time[findthis])
#obspy.UTCDateTime(tab['time'][k]).timestamp

cat = cat[5354:5933]

print('number of events, post_time filt: ' + str(len(cat)))

# df[df.Length > 7]

number of events, pre-filt: 41327
2018-06-02T03:43:36.880Z
2018-06-04T07:50:59.940Z
number of events, post_time filt: 579


In [53]:
# FILTER BY AREA ! 

# by lat,lon ! 
# lat_max = 19.45502
# lat_min = 19.36596 
# lon_max = -155.20426 
# lon_min = -155.32703

lat_min = 19.365964975838335
lat_max =  19.45502134426063
lon_min = -155.3270365580801
lon_max = -155.20426605913994

cat = cat[cat.latitude>=lat_min]
cat = cat[cat.latitude<=lat_max]
cat = cat[cat.longitude>=lon_min]
cat = cat[cat.longitude<=lon_max]

print('number of events, post_area filt: ' + str(len(cat)))

number of events, post_area filt: 523


In [55]:
# how many events in the data directory?  
evfilelist_f = glob.glob(DataDir+'*')
print(type(evfilelist_f))
print(len(evfilelist_f))
print(evfilelist_f[0:5])

evfilelist = []
for ev in evfilelist_f:
    ev = ev.split('/')[6] 
    ev = ev.strip('.txt')
    evfilelist.append(ev)

print(evfilelist[0:5])

evstalist = pd.DataFrame({'id':evfilelist})

<class 'list'>
41279
['/home/ben/data/Kilauea/waveforms_PUHI/hv70476572.txt', '/home/ben/data/Kilauea/waveforms_PUHI/hv70218497.txt', '/home/ben/data/Kilauea/waveforms_PUHI/hv70478067.txt', '/home/ben/data/Kilauea/waveforms_PUHI/hv70409077.txt', '/home/ben/data/Kilauea/waveforms_PUHI/hv70318106.txt']
['hv70476572', 'hv70218497', 'hv70478067', 'hv70409077', 'hv70318106']


In [56]:
## Go through catalog and ask if the event is in the evlist, 
## if not, remove it from the dataframe
## then pickle it ! 
print('N events in catalog: '+str(len(cat)))
print('N events at station '+ station_name+': '+str(len(evfilelist)))

N events in catalog: 523
N events at station PUHI: 41279


In [57]:
# MERGE catalog and list, getting rid of all events that are not in the list ! 
#pd.merge(adf, bdf,how='inner', on='x1')

cat_PUHI = pd.merge(cat,evstalist,how='inner',on='id')
print(len(cat_PUHI))
print(cat_PUHI.columns)
#cat.rename(columns = {'y':'year'})
cat_PUHI = cat_PUHI.rename(columns = {'latitude':'lat','longitude':'lon'})
print(cat_PUHI.columns)

523
Index(['index', 'time', 'latitude', 'longitude', 'depth', 'mag',
       'dist from Halemaumau (degrees)', 'id', 'type', 'horizontalError',
       'depthError', 'magError', 'starttimes'],
      dtype='object')
Index(['index', 'time', 'lat', 'lon', 'depth', 'mag',
       'dist from Halemaumau (degrees)', 'id', 'type', 'horizontalError',
       'depthError', 'magError', 'starttimes'],
      dtype='object')


In [59]:
cat_PUHI.to_pickle(catname_pkl)

In [58]:
print(catname_pkl)

./catalogs/cat_PUHI_zoomOnecycle_v2.pkl
